In [ ]:
import pandas as pd
import xlrd
import datetime
import numpy as np
import time

In [ ]:
def read_excel_workbook(path):
    workbook = xlrd.open_workbook(path)
    sheet = workbook.sheet_by_index(0)
    data = {}
    for i in range(sheet.ncols):
        # 按列读取数据，第一个数据是列的的名字,剩下的数据是列的值
        h, *rest = sheet.col_values(i)
        data[h] = rest
    frame = pd.DataFrame(data)
    return frame

In [ ]:
def ChangeDateFormat(x):
    if x.count('/')>0:
        dt = time.strptime(x,'%Y/%m/%d')
    elif x.count('-')>0:
        dt = time.strptime(x,'%Y-%m-%d')
    dtStamp = time.mktime(dt)
    dtDateTime = datetime.datetime.fromtimestamp(dtStamp)
    return dtDateTime

In [ ]:
startDateString = '2018-01-01'
startTime = time.strptime(startDateString,'%Y-%m-%d')
startTimeStamp = time.mktime(startTime)
startDateTime = datetime.datetime.fromtimestamp(startTimeStamp)

gen = list(range(180))
dateDictionary = {'date':[],'weekNum':[],'weekDay':[],'dayType':[]}
for i in range(180):
    moveOn = datetime.timedelta(days = i)
    dt = startDateTime + moveOn
    dateDictionary['date'].append(dt)
    dateDictionary['weekNum'].append(dt.isocalendar()[1])
    dateDictionary['weekDay'].append(dt.isocalendar()[2])
    if dt.isocalendar()[2] >=6:
        dateDictionary['dayType'].append('节假日')
    else:
        dateDictionary['dayType'].append('工作日')
dateDataFrame = pd.DataFrame(dateDictionary)

In [ ]:
dataSource = read_excel_workbook('/Volumes/Transcend/Blianlifeng/专题数据/果汁/datasource.xlsx')

In [ ]:
storeMapping = read_excel_workbook('/Volumes/Transcend/Blianlifeng/专题数据/果汁/store_Mapping.xlsx')
skuMapping = read_excel_workbook('/Volumes/Transcend/Blianlifeng/专题数据/果汁/sku_Mapping.xlsx')

In [ ]:
storeMapping['上架日期'] = storeMapping['上架日期'].apply(ChangeDateFormat)
storeMapping['下架日期'] = storeMapping['下架日期'].apply(ChangeDateFormat)
dataSource['order_date'] = dataSource['order_date'].apply(ChangeDateFormat)

In [ ]:
dataSource = pd.merge(dataSource,dateDataFrame,how='inner',on=None,left_on='order_date',right_on='date')

In [ ]:
dataSource = pd.merge(dataSource,storeMapping[['store_code','批次']],how='inner',on='store_code')

In [ ]:
dataSource.drop(['type','date'],axis='columns',inplace=True)

In [ ]:
dateDataFrame.head()

In [ ]:
dataSource.head()

In [ ]:
storeMapping.head()

In [ ]:
store_count = []
for i in dateDataFrame['date']:
    c = len(storeMapping[(storeMapping['上架日期']<=i) & (storeMapping['下架日期']>=i)])
    store_count.append(c)
dateDataFrame['storeCount'] = store_count

In [ ]:
sum1 = dataSource[dataSource['dayType']=='工作日'][['cost_total','sku_cnt','payable_price_total']].groupby([dataSource['weekNum'],dataSource['order_date'],dataSource['sku_name']]).sum()
sum1 = pd.merge(sum1.reset_index(),dateDataFrame[['date','storeCount']],how='inner',left_on='order_date',right_on='date')
sum1['avgSkuCnt'] = sum1['sku_cnt']/sum1['storeCount']
sum1['avgCost'] = sum1['cost_total']/sum1['storeCount']
sum1['avgPayPriceTotal'] = sum1['payable_price_total']/sum1['storeCount']
sum2 = sum1[['avgSkuCnt','avgPayPriceTotal','avgCost']].groupby([sum1['weekNum'],sum1['sku_name']]).mean()
sum2['avgProfit'] = sum2['avgPayPriceTotal'] - sum2['avgCost']
sum2['avgProfitRate'] = sum2['avgProfit']/sum2['avgPayPriceTotal']
float_format = lambda x : '%.2f' % x
percent_format = lambda x :'%.2f%%' % (x * 100)
sum2[['avgSkuCnt','avgCost','avgPayPriceTotal','avgProfit']] = sum2[['avgSkuCnt','avgCost','avgPayPriceTotal','avgProfit']].applymap(float_format)
sum2['avgProfitRate'] = sum2['avgProfitRate'].map(percent_format)
sum2